In [1]:
!git clone https://github.com/facebookresearch/demucs.git

Cloning into 'demucs'...
remote: Enumerating objects: 5457, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 5457 (delta 125), reused 190 (delta 107), pack-reused 5204
Receiving objects: 100% (5457/5457), 76.62 MiB | 23.81 MiB/s, done.
Resolving deltas: 100% (861/861), done.


In [2]:
%cd demucs/

/content/demucs


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring soundfile: markers 'sys_platform == "win32"' don't match your environment
     |████████████████████████████████| 87 kB 1.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 393 kB 10.1 MB/s 
     |████████████████████████████████| 61 kB 233 kB/s 
     |████████████████████████████████| 151 kB 19.8 MB/s 
     |████████████████████████████████| 59 kB 3.3 MB/s 
     |████████████████████████████████| 190 kB 19.2 MB/s 
     |████████████████████████████████| 15.1 MB 13.4 MB/s 
     |████████████████████████████████| 46 kB 4.8 MB/s 
     |████████████████████████████████| 73 kB 2.0 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 117 kB 68.2 MB/s 
     |████████████████████████████████| 66 kB 5.4 MB/s 
     |███

In [4]:
!pip install nussl
!pip install git+https://github.com/source-separation/tutorial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 239 kB 7.6 MB/s 
     |████████████████████████████████| 185 kB 49.1 MB/s 
     |████████████████████████████████| 264 kB 49.9 MB/s 
     |████████████████████████████████| 3.3 MB 47.6 MB/s 
     |████████████████████████████████| 51 kB 76 kB/s 
     |████████████████████████████████| 361 kB 42.8 MB/s 
     |████████████████████████████████| 90 kB 10.7 MB/s 
     |████████████████████████████████| 6.6 MB 22.9 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=fcc09d31c40b0744c6c5cc421390cd30d5f30e451dc9770a9c81a690c71d3a01
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for jams: filename=jams-0.3.4-py3-none-any.whl size=64922 sha256=234acdd1e402c85443a6b40d641ef7dd66e1952a4f1edde5727920e0f03131e4
  Stored in directory: /root/.cache/pip/wh

In [5]:
import nussl
from IPython.display import Audio
import IPython
from common import viz

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [6]:
# Run this command to download X 7-second clips from MUSDB18
musdb = nussl.datasets.MUSDB18(download=True)

Done!


In [7]:
musdb_train = nussl.datasets.MUSDB18(subsets=['train'])
musdb_test = nussl.datasets.MUSDB18(subsets=['test'])

# Extract data to file system

In [8]:
from pathlib import Path

# create train folder
train_folder = Path('../mini_musdb/train/').expanduser()  
train_folder.mkdir(parents=True, exist_ok=True)  

In [9]:
# create test folder
test_folder = Path('../mini_musdb/test/').expanduser()  
test_folder.mkdir(parents=True, exist_ok=True)  

In [10]:
# For each item (track) in the train set, iterate over its sources (stems),
# create a folder for the stem if it doesn't exist already (drums, bass, vocals, other) 
# and place the stem audio file in this folder, using the song name as the filename
for item in musdb_train:
    song_name = item['mix'].file_name
    for key, val in item['sources'].items():
        src_path = train_folder / song_name 
        src_path.mkdir(exist_ok=True)
        src_path = str(src_path / key) + '.wav'
        val.write_audio_to_file(src_path)

In [11]:
for item in musdb_test:
    song_name = item['mix'].file_name
    for key, val in item['sources'].items():
        src_path = test_folder / song_name 
        src_path.mkdir(exist_ok=True)
        src_path = str(src_path / key) + '.wav'
        val.write_audio_to_file(src_path)

In [12]:
# Save raw mixture file in directories
for item in musdb_train:
    song_name = item['mix'].file_name
    src_path = train_folder / song_name 
    src_path.mkdir(exist_ok=True)
    src_path = str(src_path / 'mixture') + '.wav'
    item['mix'].write_audio_to_file(src_path)

In [13]:
for item in musdb_test:
    song_name = item['mix'].file_name
    src_path = test_folder / song_name 
    src_path.mkdir(exist_ok=True)
    src_path = str(src_path / 'mixture') + '.wav'
    item['mix'].write_audio_to_file(src_path)

# Create mixture files

In [14]:
from demucs.wav import Wavset, build_metadata

In [15]:
# Root folder for the dataset
root = Path("../mini_musdb/")
train_path = Path("../mini_musdb/train/")
test_path = Path("../mini_musdb/test/")

# Build metadata for the Wavset
sources=["bass", "drums", "other", "vocals"]
ext=".wav"
metadata_train = build_metadata(path=train_path, sources=sources, ext=ext)
metadata_test = build_metadata(path=test_path, sources=sources, ext=ext)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 148.78it/s]


In [16]:
train_set = Wavset(train_path, metadata_train, sources,
                       segment=11, shift=1,
                       samplerate=44100, channels=2,
                       normalize=True)

test_set = Wavset(test_path, metadata_test, sources,
                       segment=11, shift=1,
                       samplerate=44100, channels=2,
                       normalize=True)

In [17]:
from demucs.demucs import Demucs
import numpy as np
import torch

In [18]:
model = Demucs(sources=['bass', 'drums', 'other', 'vocals'], audio_channels=2)

In [19]:
sample = train_set[0]

In [20]:
sample = musdb_train[0]['mix'].audio_data
sample_tensor = torch.from_numpy(sample.astype(dtype=np.float32))

In [21]:
mini_sample = sample_tensor[:,:80000]

In [22]:
mini_sample = torch.unsqueeze(mini_sample, 0)

In [23]:
mini_sample.shape

torch.Size([1, 2, 80000])

In [24]:
output = model(mini_sample)
output_arr = output.detach().numpy()

In [26]:
sample_rate = musdb_test[0]['sources']['drums'].sample_rate
audio_player = Audio(data=output_arr[0,3,:,:], rate=sample_rate)
IPython.display.display(audio_player)

# Train model